Use the De Casteljau algorithm with SLERP on 4 given rotations.

Re-parameterize by rotation angle (i.e. twice the arc-length).

In [ ]:
from IPython.display import HTML

In [ ]:
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from unit_quaternion import Quaternion, UnitQuaternion, slerp

In [ ]:
from splines import ConstantSpeedAdapter

In [ ]:
from helper import angles2quat, animate_rotations

In [ ]:
class BezierSegment:
    
    def __init__(self, q0, q1, q2, q3):
        self.q0 = q0
        self.q1 = q1
        self.q2 = q2
        self.q3 = q3
        self.grid = [0, 1]
        
    def evaluate(self, t, n=0):
        if not np.isscalar(t):
            return np.array([self.evaluate(t, n=n) for t in t])
        if not 0 <= t <= 1:
            raise ValueError(f'invalid t: {t}')
        if n == 0:
            slerp_1_2 = slerp(self.q1, self.q2, t)
            return slerp(
                slerp(slerp(self.q0, self.q1, t), slerp_1_2, t),
                slerp(slerp_1_2, slerp(self.q2, self.q3, t), t),
                t)
        elif n == 1:
            slerp_1_2 = slerp(self.q1, self.q2, t)
            one = slerp(slerp(self.q0, self.q1, t), slerp_1_2, t)
            two = slerp(slerp_1_2, slerp(self.q2, self.q3, t), t)
            x, y, z = (two * one.inverse()).log_map()
            # NB: twice the angle, times 3 because degree 3
            return x * 2 * 3, y * 2 * 3, z * 2 * 3
        else:
            raise NotImplementedError(f'invalid n: {n}')

In [ ]:
b1 = BezierSegment(
    angles2quat(0, 0, 0),
    angles2quat(10, 0, 0),
    angles2quat(80, 0, 0),
    angles2quat(90, 0, 0),
)

In [ ]:
b1.grid

In [ ]:
b1.evaluate(0), b1.evaluate(0.5), b1.evaluate(1)

In [ ]:
b1.evaluate(0, 1)

In [ ]:
b1.evaluate(0.5, 1)

In [ ]:
b1.evaluate(1, 1)

In [ ]:
b2 = ConstantSpeedAdapter(b1)

In [ ]:
b2.grid

In [ ]:
b2.evaluate(0)

In [ ]:
b2.evaluate(b2.grid)

In [ ]:
def generate_rotations(s):
    return s.evaluate(np.linspace(s.grid[0], s.grid[-1], 100))

In [ ]:
ani = animate_rotations({
    'non-constant speed': generate_rotations(b1),
    'constant speed': generate_rotations(b2),
}, figsize=(6, 3), interval=30)
display(HTML(ani.to_jshtml(default_mode='reflect')))

In [ ]:
b3 = BezierSegment(
    angles2quat(0, 0, 0),
    angles2quat(0, 90, 0),
    angles2quat(90, -45, 0),
    angles2quat(90, 0, 0),
)

In [ ]:
b4 = ConstantSpeedAdapter(b3)

In [ ]:
ani = animate_rotations({
    'non-constant speed': generate_rotations(b3),
    'constant speed': generate_rotations(b4),
}, figsize=(6, 3), interval=30)
display(HTML(ani.to_jshtml(default_mode='reflect')))

In [ ]:
b5 = BezierSegment(
    angles2quat(0, 0, 0),
    angles2quat(10, 0, -179),
    angles2quat(20, 90, 179),
    angles2quat(30, 90, 0),
)

In [ ]:
b6 = ConstantSpeedAdapter(b5)

In [ ]:
ani = animate_rotations({
    'non-constant speed': generate_rotations(b5),
    'constant speed': generate_rotations(b6),
}, figsize=(6, 3), interval=30)
display(HTML(ani.to_jshtml(default_mode='reflect')))

## Bézier to Hermite

In [ ]:
q1 = angles2quat(0, 0, 0)
q2 = angles2quat(90, 0, 0)
q3 = angles2quat(180, 0, 0)
q4 = angles2quat(180, 90, 0)
b7 = BezierSegment(q1, q2, q3, q4)

start velocity:

In [ ]:
b7.evaluate(0, 1)

4.71 radians per second = 0.75 rotations per second

end velocity:

In [ ]:
b7.evaluate(1, 1)

In [ ]:
chord1 = q2 * q1.inverse()

In [ ]:
chord1.axis

In [ ]:
chord1.angle

In [ ]:
_ * 3

In [ ]:
chord3 = q4 * q3.inverse()

In [ ]:
chord3.axis

In [ ]:
chord3.angle

## Hermite to Bézier

In [ ]:
import math

In [ ]:
q0 = angles2quat(-90, 0, -90)
v0 = 0, -math.pi/2, 0
v1 = math.pi/2, 0, 0
q1 = angles2quat(180, 0, 0)

In [ ]:
chord0 = UnitQuaternion.from_axis_angle((0, 1, 0), (1/3) * 0.1)

In [ ]:
q0_hat = chord0 * q0

In [ ]:
chord1 = UnitQuaternion.from_axis_angle((1, 0, 0), (1/3) * 0.1)

In [ ]:
q1_hat = (q1.inverse() * chord1).inverse()

In [ ]:
b8 = BezierSegment(q0, q0_hat, q1_hat, q1)

In [ ]:
b8.evaluate(0, 1)

In [ ]:
b8.evaluate(1, 1)

In [ ]:
ani = animate_rotations(generate_rotations(b8), interval=30)
display(HTML(ani.to_jshtml(default_mode='reflect')))